In [33]:
from models.stage2.transformers import BidirectionalTransformer
from models.stage1.encoder_decoder import VQVAEEncoder, VQVAEDecoder
from models.stage1.vq import VectorQuantize


In [37]:
import torch
import numpy as np
from preprocessing.data_pipeline import build_data_pipeline

from preprocessing.preprocess_ucr import UCRDatasetImporter
from pathlib import Path

from models.stage2.sample import unconditional_sample, conditional_sample

# from evaluation.evaluation import Evaluation
from utils import get_root_dir, load_yaml_param_settings, save_model
from models.stage2.maskgit import MaskGIT
import os
import tempfile
from evaluation.model_eval import Evaluation
import matplotlib.pyplot as plt


In [40]:
config_dir = "configs/config.yaml"
config = load_yaml_param_settings(config_dir)
dataset_name = config["dataset"]["dataset_name"]

dataset_importer = UCRDatasetImporter(**config["dataset"])
batch_size = config["dataset"]["batch_sizes"]["stage2"]
train_data_loader, test_data_loader = [
    build_data_pipeline(batch_size, dataset_importer, config, kind)
    for kind in ["train", "test"]
]

input_length = train_data_loader.dataset.X.shape[-1]
n_classes = len(np.unique(train_data_loader.dataset.Y.flatten()))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


dim = config["encoder"]["dim"]
in_channels = config["dataset"]["in_channels"]
downsampled_width = config["encoder"]["downsampled_width"]
n_fft = config["VQVAE"]["n_fft"]

downsample_rate = 1

# def compute_downsample_rate(input_length: int, n_fft: int, downsampled_width: int):
#     return (
#         round(input_length / (np.log2(n_fft) - 1) / downsampled_width)
#         if input_length >= downsampled_width
#         else 1
#     )
# downsample_rate = compute_downsample_rate(
#     input_length, n_fft, downsampled_width
# )


self.X_train.shape: (1000, 1024)
self.X_test.shape: (8236, 1024)
# unique labels (train): [0 1 2]
# unique labels (test): [0 1 2]


In [35]:
from stage2 import load_args


In [41]:
encoder = VQVAEEncoder(
    dim, 2 * in_channels, downsample_rate, config["encoder"]["n_resnet_blocks"]
)
decoder = VQVAEDecoder(
    dim, 2 * in_channels, downsample_rate, config["decoder"]["n_resnet_blocks"]
)
vq_model = VectorQuantize(
    dim, config["VQVAE"]["codebook"]["size"], **config["VQVAE"]
)

1


In [47]:
decoder.load_state_dict(torch.load('saved_models_old/decoder-StarLightCurves.ckpt'))




RuntimeError: Error(s) in loading state_dict for VQVAEDecoder:
	Missing key(s) in state_dict: "decoder.4.weight", "decoder.4.bias", "decoder.5.weight", "decoder.5.bias". 
	Unexpected key(s) in state_dict: "decoder.6.block.0.weight", "decoder.6.block.0.bias", "decoder.6.block.1.weight", "decoder.6.block.1.bias", "decoder.6.block.1.running_mean", "decoder.6.block.1.running_var", "decoder.6.block.1.num_batches_tracked", "decoder.7.block.0.weight", "decoder.7.block.0.bias", "decoder.7.block.1.weight", "decoder.7.block.1.bias", "decoder.7.block.1.running_mean", "decoder.7.block.1.running_var", "decoder.7.block.1.num_batches_tracked", "decoder.8.block.0.weight", "decoder.8.block.0.bias", "decoder.8.block.1.weight", "decoder.8.block.1.bias", "decoder.8.block.1.running_mean", "decoder.8.block.1.running_var", "decoder.8.block.1.num_batches_tracked", "decoder.9.weight", "decoder.9.bias", "decoder.10.weight", "decoder.10.bias", "decoder.4.block.0.weight", "decoder.4.block.0.bias", "decoder.4.bloc

In [46]:
encoder.load_state_dict(torch.load('saved_models_old/encoder-StarLightCurves.ckpt'))

# def load(model, dirname, fname):
#     """
#     model: instance
#     path_to_saved_model_fname: path to the ckpt file (i.e., trained model)
#     """
#     print(dirname)
#     print(fname)
#     try:
#         model.load_state_dict(torch.load(dirname.joinpath(fname)))
#     except FileNotFoundError:
#         dirname = Path(tempfile.gettempdir())
#         model.load_state_dict(torch.load(dirname.joinpath(fname)))

RuntimeError: Error(s) in loading state_dict for VQVAEEncoder:
	Missing key(s) in state_dict: "encoder.1.0.convs.1.weight", "encoder.1.0.convs.1.bias", "encoder.1.0.convs.2.weight", "encoder.1.0.convs.2.bias", "encoder.1.0.convs.2.running_mean", "encoder.1.0.convs.2.running_var", "encoder.1.0.convs.5.weight", "encoder.1.0.convs.5.bias", "encoder.1.1.weight", "encoder.1.1.bias", "encoder.1.1.running_mean", "encoder.1.1.running_var", "encoder.2.0.convs.1.weight", "encoder.2.0.convs.1.bias", "encoder.2.0.convs.2.weight", "encoder.2.0.convs.2.bias", "encoder.2.0.convs.2.running_mean", "encoder.2.0.convs.2.running_var", "encoder.2.0.convs.5.weight", "encoder.2.0.convs.5.bias", "encoder.2.1.weight", "encoder.2.1.bias", "encoder.2.1.running_mean", "encoder.2.1.running_var", "encoder.3.0.convs.1.weight", "encoder.3.0.convs.1.bias", "encoder.3.0.convs.2.weight", "encoder.3.0.convs.2.bias", "encoder.3.0.convs.2.running_mean", "encoder.3.0.convs.2.running_var", "encoder.3.0.convs.5.weight", "enco